---
title: "Analisi della copertura del suolo in Italia dal 1990 al 2018 usando la CORINE Land Cover"
description: "Vediamo come è cambiata la copertura del suolo in Italia dal 1990 al 2018 usando i dati della CORINE Land Cover."
lang: it
date: "2022 08 26"
open-graph:
    title: "Analisi della copertura del suolo in Italia dal 1990 al 2018 usando la CORINE Land Cover"
    description: "Vediamo come è cambiata la copertura del suolo in Italia dal 1990 al 2018 usando i dati della CORINE Land Cover."
    image: ../../project_logo.jpg
categories: [vector, geoprocessing]
image: ../../project_logo.jpg
format:
    html:
        toc: true
        code-fold: false
execute:
  enabled: false
---

Qualche anno fa analizzai i dati del progetto [CORINE Land Cover](https://land.copernicus.eu/pan-european/corine-land-cover) relativi alla [Campania](https://massimilianomoraca.it/blog/gis/luso-del-suolo-in-campania-dati-dal-progetto-corine-land-cover/), focalizzandomi sui tre censimenti tra il 2000 ed il 2012. Oggi invece estenderò quell'analisi inserendo sia i dati del 1990 che gli ultimi disponibili attualmente, cioè quelli del 2018.

L'analisi questa volta la farò usando Python e la estenderò al resto dell'Italia verso la fine.

## Prima di iniziare


### Librerie
Per raggiungere l'obiettivo userò le librerie che seguono:

In [1]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### Fonti dati
Ho rielaborato i dati CORINE Land Cover dal 1990 al 2018 inserendoli in un unico [GeoPackage](https://massimilianomoraca.it/blog/gis/il-geopackage-una-valida-alternativa-al-formato-shape/) che è possibile scaricare da [qui](https://massimilianomoraca.it/media/uploads/documents/2022/08/26/clcitalia.7z). I dati sono stati processati preventivamente usando [PostGIS](https://postgis.net/) in modo da associare ai poligoni della CLC anche quelli dei Comuni italiani al 2022 dell'[ISTAT](https://www.istat.it/it/archivio/222527). E' stato indispensabile questo passaggio preventivo perchè ho dovuto correggere gli errori topologici sui dati ISTAT e, al momento, trovo molto più comodo [correggere gli errori topologici](https://massimilianomoraca.it/blog/gis/correggere-le-geometrie-non-valide/) tramite SQL piuttosto che con Python.

In [2]:
main_folder = Path('/home/max/Downloads/clcitalia')
dataset = main_folder.joinpath('clcitalia.gpkg')

# 1. Lettura e normalizzazione dei dati
In questa fase ho proceduto ad estrarre un po' di dati statistici che userò nei passaggi successivi. Sono perlopiu analisi mirate al calcolo delle superfici che ho espresso in [ettari](https://it.wikipedia.org/wiki/Ettaro). Fatte queste analisi mi sono liberato dei dati geometrici perchè, non dovendoli più usare, mi alleggeriscono le analisi successive in termini di prestazioni hardware.

In [ ]:
# CLC 1990
clc1990 = gpd.read_file(dataset, layer='clc1990')
clc1990.insert(loc=10, column='sup_com_ha', value=clc1990.sup_com_mq / 10000)
clc1990.insert(loc=11, column='surface_ha_1990', value=round(clc1990.geometry.area / 10000, 4))
clc1990.drop(columns={'sup_com_mq', 'geometry'}, inplace=True)

clc1990.head(10)

In [ ]:
# CLC 2000
clc2000 = gpd.read_file(dataset, layer='clc2000')
clc2000.insert(loc=10, column='sup_com_ha', value=clc2000.sup_com_mq / 10000)
clc2000.insert(loc=11, column='surface_ha_2000', value=round(clc2000.geometry.area / 10000, 4))
clc2000.drop(columns={'sup_com_mq', 'geometry'}, inplace=True)

clc2000.head(10)

In [ ]:
# CLC 2006
clc2006 = gpd.read_file(dataset, layer='clc2006')
clc2006.insert(loc=10, column='sup_com_ha', value=clc2006.sup_com_mq / 10000)
clc2006.insert(loc=11, column='surface_ha_2006', value=round(clc2006.geometry.area / 10000, 4))
clc2006.drop(columns={'sup_com_mq', 'geometry'}, inplace=True)

clc2006.head(10)

In [ ]:
# CLC 2012
clc2012 = gpd.read_file(dataset, layer='clc2012')
clc2012.insert(loc=10, column='sup_com_ha', value=clc2012.sup_com_mq / 10000)
clc2012.insert(loc=11, column='surface_ha_2012', value=round(clc2012.geometry.area / 10000, 4))
clc2012.drop(columns={'sup_com_mq', 'geometry'}, inplace=True)

clc2012.head(10)

In [3]:
# CLC 2018
clc2018 = gpd.read_file(dataset, layer='clc2018')
clc2018.insert(loc=10, column='sup_com_ha', value=clc2018.sup_com_mq / 10000)
clc2018.insert(loc=11, column='surface_ha_2018', value=round(clc2018.geometry.area / 10000, 4))
clc2018.drop(columns={'sup_com_mq', 'geometry'}, inplace=True)

clc2018.head(10)

KeyboardInterrupt: 

Le colonne `label*` fanno riferimento alla nomenclatura delle tre classi della mappa CORINE Land Cover per ogni singolo anno. Per interpretare la nomenclatura vai [qui](https://land.copernicus.eu/user-corner/technical-library/corine-land-cover-nomenclature-guidelines/html/index.html). Come puoi vedere sono presenti le colonne relative al Comune, Provincia e Regione di apparteneza, l'estensione dell'area comunale(`sup_com_ha`) e quelle relative alle superfici della CLC che mi sono calcolato in precedenza.

# 2. Focus sulla Campania
Per appartenza territoriale partirò proprio dalla Campania, lo faccio anche per riprendere parte della analisi del mio vecchio articolo.

In [ ]:
clc1990_campania = clc1990[clc1990['regione'] == 'Campania']
clc2000_campania = clc2000[clc2000['regione'] == 'Campania']
clc2006_campania = clc2006[clc2006['regione'] == 'Campania']
clc2012_campania = clc2012[clc2012['regione'] == 'Campania']
clc2018_campania = clc2018[clc2018['regione'] == 'Campania']

Test di visualizzazione

In [ ]:
clc2000_campania

### 2.1.1 Stato al 1990
Come primo step mi sono concentrato sui dati del 1990; ho pulito un po' la tabella per rendere meglio comprensibile la sua lettura ed ho estratto valori per tutte e tre le classi. Quello che ho fatto per il 1990 l'ho ripetuto per i restanti anni.

**Livello 1**

In [ ]:
campania_clc1990_label1 = pd.DataFrame(clc1990_campania.groupby('label1')['surface_ha_1990'].sum())
campania_clc1990_label1.reset_index(inplace=True)
campania_clc1990_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)
campania_clc1990_label1

**Livello 2**

In [ ]:
campania_clc1990_label2 = pd.DataFrame(clc1990_campania.groupby('label2')['surface_ha_1990'].sum())
campania_clc1990_label2.reset_index(inplace=True)
campania_clc1990_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)
campania_clc1990_label2

**Livello 3**

In [ ]:
campania_clc1990_label3 = pd.DataFrame(clc1990_campania.groupby('label3')['surface_ha_1990'].sum())
campania_clc1990_label3.reset_index(inplace=True)
campania_clc1990_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)
campania_clc1990_label3

### 2.1.2 Stato al 2000

In [ ]:
campania_clc2000_label1 = pd.DataFrame(clc2000_campania.groupby('label1')['surface_ha_2000'].sum())
campania_clc2000_label1.reset_index(inplace=True)
campania_clc2000_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)

campania_clc2000_label2 = pd.DataFrame(clc2000_campania.groupby('label2')['surface_ha_2000'].sum())
campania_clc2000_label2.reset_index(inplace=True)
campania_clc2000_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)

campania_clc2000_label3 = pd.DataFrame(clc2000_campania.groupby('label3')['surface_ha_2000'].sum())
campania_clc2000_label3.reset_index(inplace=True)
campania_clc2000_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)

### 2.1.3 Stato al 2006

In [ ]:
campania_clc2006_label1 = pd.DataFrame(clc2006_campania.groupby('label1')['surface_ha_2006'].sum())
campania_clc2006_label1.reset_index(inplace=True)
campania_clc2006_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)

campania_clc2006_label2 = pd.DataFrame(clc2006_campania.groupby('label2')['surface_ha_2006'].sum())
campania_clc2006_label2.reset_index(inplace=True)
campania_clc2006_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)

campania_clc2006_label3 = pd.DataFrame(clc2006_campania.groupby('label3')['surface_ha_2006'].sum())
campania_clc2006_label3.reset_index(inplace=True)
campania_clc2006_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)

### 2.1.4 Stato al 2012

In [ ]:
campania_clc2012_label1 = pd.DataFrame(clc2012_campania.groupby('label1')['surface_ha_2012'].sum())
campania_clc2012_label1.reset_index(inplace=True)
campania_clc2012_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)

campania_clc2012_label2 = pd.DataFrame(clc2012_campania.groupby('label2')['surface_ha_2012'].sum())
campania_clc2012_label2.reset_index(inplace=True)
campania_clc2012_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)

campania_clc2012_label3 = pd.DataFrame(clc2012_campania.groupby('label3')['surface_ha_2012'].sum())
campania_clc2012_label3.reset_index(inplace=True)
campania_clc2012_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)

### 2.1.5 Stato al 2018

In [ ]:
campania_clc2018_label1 = pd.DataFrame(clc2018_campania.groupby('label1')['surface_ha_2018'].sum())
campania_clc2018_label1.reset_index(inplace=True)
campania_clc2018_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)

campania_clc2018_label2 = pd.DataFrame(clc2018_campania.groupby('label2')['surface_ha_2018'].sum())
campania_clc2018_label2.reset_index(inplace=True)
campania_clc2018_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)

campania_clc2018_label3 = pd.DataFrame(clc2018_campania.groupby('label3')['surface_ha_2018'].sum())
campania_clc2018_label3.reset_index(inplace=True)
campania_clc2018_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)

## 2.2 Grafici
E' il momento di approfondire questi dati!
Ho estratto grafici per tutte e tre le classi ed ho cercato di dare una interpretazione del perchè di alcuni fenomeni.

### 2.2.1 Grafici per il livello 1

In [ ]:
lvl1 = [campania_clc1990_label1, campania_clc2000_label1, campania_clc2006_label1, campania_clc2012_label1, campania_clc2018_label1]
lvl1_df = reduce(lambda  left,right: pd.merge(left,right,on=['CORINE Land Cover - level 1'], how='outer'), lvl1)
lvl1_df.fillna(0, inplace=True)
lvl1_df.insert(loc=6, column='Surf.Diff. | 2018-1990', value=(lvl1_df.surface_ha_2018 - lvl1_df.surface_ha_1990))
lvl1_df.set_index('CORINE Land Cover - level 1', inplace=True)
lvl1_df.rename(columns={'surface_ha_1990': '1990', 'surface_ha_2000': '2000', 'surface_ha_2006': '2006', 'surface_ha_2012': '2012', 'surface_ha_2018': '2018'}, inplace=True)
lvl1_df.sort_index(ascending=True, inplace=True)
lvl1_df

In [ ]:
fig1 = plt.figure(figsize=(20, 10), dpi=100)
sub_fig = GridSpec(nrows=1, ncols=1, figure=fig1, hspace=0.35)
ax = fig1.add_subplot(sub_fig[0, 0])
lvl1_df.plot.bar(ax=ax, rot=0)

ax.yaxis.set_label_text('ha')
ax.set_xlabel('')
ax.grid(color='lightgrey', linestyle='dashdot')

plt.legend(loc='best')

E' visibile abbastanza chiaramente un netto aumento delle superfici artificiali con un conseguente arretramento di quelle forestate e di quelle agricole. Le aree forestate tra il 1990 ed il 2018 hanno subito un arretramento quasi doppio rispetto a quelle agricole e le superfici artificiali sono quelle che hanno maggiormente *"beneficiato"* di questo arretramento complessivo delle due classi di CLC. Si potrebbe dire che l'urbanizzato ha eroso foreste ed aree agricole in quasi 30 anni.

### 2.2.2 Grafici per il livello 2

In [ ]:
lvl2 = [campania_clc1990_label2, campania_clc2000_label2, campania_clc2006_label2, campania_clc2012_label2, campania_clc2018_label2]
lvl2_df = reduce(lambda  left,right: pd.merge(left,right,on=['CORINE Land Cover - level 2'], how='outer'), lvl2)
lvl2_df.fillna(0, inplace=True)
lvl2_df.insert(loc=6, column='Surf.Diff. | 2000-1990', value=(lvl2_df.surface_ha_2000 - lvl2_df.surface_ha_1990))
lvl2_df.insert(loc=7, column='Surf.Diff. | 2006-2000', value=(lvl2_df.surface_ha_2006 - lvl2_df.surface_ha_2000))
lvl2_df.insert(loc=8, column='Surf.Diff. | 2012-2006', value=(lvl2_df.surface_ha_2012 - lvl2_df.surface_ha_2006))
lvl2_df.insert(loc=9, column='Surf.Diff. | 2018-2012', value=(lvl2_df.surface_ha_2018 - lvl2_df.surface_ha_2012))
lvl2_df.insert(loc=9, column='Surf.Diff. | 2018-1990', value=(lvl2_df.surface_ha_2018 - lvl2_df.surface_ha_1990))
lvl2_df.set_index('CORINE Land Cover - level 2', inplace=True)
lvl2_df = lvl2_df[['Surf.Diff. | 2000-1990', 'Surf.Diff. | 2006-2000', 'Surf.Diff. | 2012-2006', 'Surf.Diff. | 2018-2012', 'Surf.Diff. | 2018-1990']]
lvl2_df.sort_index(ascending=True, inplace=True)
lvl2_df

In [ ]:
fig2 = plt.figure(figsize=(20, 10), dpi=100)
sub_fig = GridSpec(nrows=1, ncols=1, figure=fig2, hspace=0.35)
ax = fig2.add_subplot(sub_fig[0, 0])
lvl2_df.plot.bar(ax=ax, rot=0)

ax.yaxis.set_label_text('ha')
ax.set_xlabel('')
ax.grid(color='lightgrey', linestyle='dashdot')

plt.xticks(rotation = 90)
plt.legend(loc='best')

Scendendo di un livello, quindi andando ad approfondire di più il dato passando al secondo livello CLC, possiamo verificare i trend negativi a quali specifiche superfici fanno riferimento.

Le aree arbustive e con alberi radi hanno avuto la peggio, a seguire i terreni arabili, pascoli e foreste. Netto è invece l'aumento delle aree aperte, classe in cui nel terzo livello CLC ci sono le aree percorse dal fuoco(da tenere in mente per richiami successivi!). Seguono, nell'incremento, le aree urbanizzate, le aree per colture eterogenee e quelle per lo sviluppo di siti industriali e commerciali.



### 2.2.3 Grafici per il livello 3

In [ ]:
lvl3 = [campania_clc1990_label3, campania_clc2000_label3, campania_clc2006_label3, campania_clc2012_label3, campania_clc2018_label3]
lvl3_df = reduce(lambda  left,right: pd.merge(left,right,on=['CORINE Land Cover - level 3'], how='outer'), lvl3)
lvl3_df.fillna(0, inplace=True)
lvl3_df.insert(loc=6, column='Surf.Diff. | 2000-1990', value=(lvl3_df.surface_ha_2000 - lvl3_df.surface_ha_1990))
lvl3_df.insert(loc=7, column='Surf.Diff. | 2006-2000', value=(lvl3_df.surface_ha_2006 - lvl3_df.surface_ha_2000))
lvl3_df.insert(loc=8, column='Surf.Diff. | 2012-2006', value=(lvl3_df.surface_ha_2012 - lvl3_df.surface_ha_2006))
lvl3_df.insert(loc=9, column='Surf.Diff. | 2018-2012', value=(lvl3_df.surface_ha_2018 - lvl3_df.surface_ha_2012))
lvl3_df.insert(loc=10, column='Surf.Diff. | 2018-1990', value=(lvl3_df.surface_ha_2018 - lvl3_df.surface_ha_1990))
lvl3_df.set_index('CORINE Land Cover - level 3', inplace=True)

lvl3_df.sort_index(ascending=True, inplace=True)
lvl3_df

In [ ]:
fig3 = plt.figure(figsize=(20, 10), dpi=100)
sub_fig = GridSpec(nrows=1, ncols=1, figure=fig3, hspace=0.35)
ax = fig3.add_subplot(sub_fig[0, 0])
lvl3_df['Surf.Diff. | 2018-1990'].plot.bar(ax=ax, rot=0)

ax.yaxis.set_label_text('ha')
ax.set_xlabel('')
ax.grid(color='lightgrey', linestyle='dashdot')

plt.xticks(rotation = 90)
plt.legend(loc='best')

Passando al livello tre della CLC, ho voluto graficare la sola differenza tra 1990 e 2018 per motivi di leggibilità del grafico. In termini di aumento di copertura del suolo c'è l'avanzamento, quasi identico, delle piccole aree destinate a colture annuali e delle aree con vegetazione frammentata. A seguire ci sono le aree urbane discontinue e quelle occupate da orti, vigneti e alberi da frutto.

**Una riflessione che mi sento di fare è che l'avanzamento del tessuto urbano discontinuo ha frazionato il territorio agricolo, generando aree a piccole colture agricole.**

Significativo anche l'aumento di estensione delle aree percorse dal fuoco.

I decrementi sono la cartina di tornasole degli incrementi che ho descritto poco fa e che potrebbero confermare la mia riflessione sul frazionamento del territorio naturale ed agricolo, portato avanti dall'avanzamento delle classi legati ai *suoli artificiali*.

## 2.3 Focus sui Comuni
L'analisi che segue tenta di rispondere alla domanda: ***"quali sono i Comuni campani in cui i fenomeni esposti poco fa hanno avuto un maggior impatto?"***

Mi concentrerò sul terzo livello della CLC, in particolare sulle aree artificiali, aree agricole e foreste ma anche sulle discariche, le aree percorse dal fuoco ed il verde urbano.

Visto che dal 2000 in poi l'accuratezza geometrica del dato CLC è aumentata considerevolmente, le analisi di dettaglio che seguono si concentreranno sulla differenza tra 2000 e 2018.

In [ ]:

# Dati 1990
focus_1990 = clc1990_campania[['pro_com', 'comune', 'provincia', 'sup_com_ha', 'label3', 'surface_ha_1990']]
focus_1990.set_index('label3', inplace=True)
focus_1990_group = pd.DataFrame(focus_1990.groupby(['pro_com', 'comune', 'sup_com_ha', 'label3'])['surface_ha_1990'].sum())
focus_1990_group.reset_index(inplace=True)

# Dati 2000
focus_2000 = clc2000_campania[['pro_com', 'comune', 'provincia', 'sup_com_ha', 'label3', 'surface_ha_2000']]
focus_2000.set_index('label3', inplace=True)
focus_2000_group = pd.DataFrame(focus_2000.groupby(['pro_com', 'comune', 'sup_com_ha', 'label3'])['surface_ha_2000'].sum())
focus_2000_group.reset_index(inplace=True)

# Dati 2006
focus_2006 = clc2006_campania[['pro_com', 'comune', 'provincia', 'sup_com_ha', 'label3', 'surface_ha_2006']]
focus_2006.set_index('label3', inplace=True)
focus_2006_group = pd.DataFrame(focus_2006.groupby(['pro_com', 'comune', 'sup_com_ha', 'label3'])['surface_ha_2006'].sum())
focus_2006_group.reset_index(inplace=True)

# Dati 2012
focus_2012 = clc2012_campania[['pro_com', 'comune', 'provincia', 'sup_com_ha', 'label3', 'surface_ha_2012']]
focus_2012.set_index('label3', inplace=True)
focus_2012_group = pd.DataFrame(focus_2012.groupby(['pro_com', 'comune', 'sup_com_ha', 'label3'])['surface_ha_2012'].sum())
focus_2012_group.reset_index(inplace=True)

# Dati 2018
focus_2018 = clc2018_campania[['pro_com', 'comune', 'provincia', 'sup_com_ha', 'label3', 'surface_ha_2018']]
focus_2018.set_index('label3', inplace=True)
focus_2018_group = pd.DataFrame(focus_2018.groupby(['pro_com', 'comune', 'sup_com_ha', 'label3'])['surface_ha_2018'].sum())
focus_2018_group.reset_index(inplace=True)

In [ ]:
focus_1990_group

### 2.3.1 Aree artificiali
I gruppi 1.1 ed 1.2 della CLC

In [ ]:

# Dati 1990
artificial_surface_1990 = focus_1990_group[focus_1990_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_1990_group = pd.DataFrame(artificial_surface_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
artificial_surface_1990_group.reset_index(inplace=True)

# Dati 2000
artificial_surface_2000 = focus_2000_group[focus_2000_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_2000_group = pd.DataFrame(artificial_surface_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
artificial_surface_2000_group.reset_index(inplace=True)

# Dati 2006
artificial_surface_2006 = focus_2006_group[focus_2006_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_2006_group = pd.DataFrame(artificial_surface_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
artificial_surface_2006_group.reset_index(inplace=True)

# Dati 2012
artificial_surface_2012 = focus_2012_group[focus_2012_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_2012_group = pd.DataFrame(artificial_surface_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
artificial_surface_2012_group.reset_index(inplace=True)

# Dati 2018
artificial_surface_2018 = focus_2018_group[focus_2018_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_2018_group = pd.DataFrame(artificial_surface_2018.groupby(['pro_com', 'comune', 'sup_com_ha'])['surface_ha_2018'].sum())
artificial_surface_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_artificial = [artificial_surface_2018_group, artificial_surface_2012_group, artificial_surface_2006_group, artificial_surface_2000_group, artificial_surface_1990_group]
focus_artificial_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_artificial)
focus_artificial_df = focus_artificial_df[focus_artificial_df['sup_com_ha'] > 0]
focus_artificial_df.fillna(0, inplace=True)
focus_artificial_df.insert(loc=3, column='perc_1990', value=round(focus_artificial_df.surface_ha_1990 / focus_artificial_df.sup_com_ha, 4) * 100)
focus_artificial_df.insert(loc=4, column='perc_2000', value=round(focus_artificial_df.surface_ha_2000 / focus_artificial_df.sup_com_ha, 4) * 100)
focus_artificial_df.insert(loc=5, column='perc_2006', value=round(focus_artificial_df.surface_ha_2006 / focus_artificial_df.sup_com_ha, 4) * 100)
focus_artificial_df.insert(loc=6, column='perc_2012', value=round(focus_artificial_df.surface_ha_2012 / focus_artificial_df.sup_com_ha, 4) * 100)
focus_artificial_df.insert(loc=7, column='perc_2018', value=round(focus_artificial_df.surface_ha_2018 / focus_artificial_df.sup_com_ha, 4) * 100)
focus_artificial_df.insert(loc=8, column='Perc.Diff. | 2018-2000', value=(focus_artificial_df.perc_2018 - focus_artificial_df.perc_2000))
focus_artificial_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
focus_artificial_df

In [ ]:
# Superfici in arretramento
reduction_artificial_area = focus_artificial_df.iloc[0:10]
reduction_artificial_area

In [ ]:
# Superfici in avanzamento
increase_artificial_area = focus_artificial_df.iloc[-10:]
increase_artificial_area.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
increase_artificial_area

In [ ]:
artificial_df = pd.concat([reduction_artificial_area, increase_artificial_area])
artificial_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
artificial_df = artificial_df[['comune', 'Perc.Diff. | 2018-2000']]
artificial_df['Perc.Diff. | 2018-2000'] = artificial_df['Perc.Diff. | 2018-2000']
artificial_df.set_index('comune', inplace=True)

fig_artificial = plt.figure(figsize=(20, 10), dpi=100)
sub_fig_artificial = GridSpec(nrows=1, ncols=1, figure=fig_artificial, hspace=0.35)
ax_artificial = fig_artificial.add_subplot(sub_fig_artificial[0, 0])
artificial_df.plot.bar(ax=ax_artificial, rot=0)

ax_artificial.yaxis.set_label_text('%')
ax_artificial.set_xlabel('')
ax_artificial.grid(color='lightgrey', linestyle='dashdot')

plt.xticks(rotation = 90)
plt.legend(loc='best')

Onestamente non so come interpretare l'arretramento delle aree artificiali, andrebbe fatta una ulteriore analisi per valutare se sono Comuni che si sono spopolati in quasi trenta anni.

L'aumento è abbastanza facile da indagare per me che sono del posto. Napoli ha una popolazione di quasi un milione di abitanti che triplica nei giorni feriali, per non parlare poi dei turisti. E' un'area molto congestionata con possibilità di nuova edilizia prossime allo zero; risulta normale quindi che Comuni più prossimi al Capoluogo di Regione stiano accogliendo "l'esubero" di popolazione. Faccio riferimento a Castello di Cisterna, Volla e Quarto.

Il caso di [Atrani](https://it.wikipedia.org/wiki/Atrani) è particolare perchè è il Comune più piccolo d'Italia ed è posto in una zona abbastanza impervia della costiera amalfitana; è probabile che nel 2000 non si sia data molta importanza all'area.

### 2.3.2 Aree forestate
Il gruppo 3.1 della CLC

In [ ]:

# Dati 1990
forest_surface_1990 = focus_1990_group[focus_1990_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_1990_group = pd.DataFrame(forest_surface_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
forest_surface_1990_group.reset_index(inplace=True)

# Dati 2000
forest_surface_2000 = focus_2000_group[focus_2000_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_2000_group = pd.DataFrame(forest_surface_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
forest_surface_2000_group.reset_index(inplace=True)

# Dati 2006
forest_surface_2006 = focus_2006_group[focus_2006_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_2006_group = pd.DataFrame(forest_surface_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
forest_surface_2006_group.reset_index(inplace=True)

# Dati 2012
forest_surface_2012 = focus_2012_group[focus_2012_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_2012_group = pd.DataFrame(forest_surface_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
forest_surface_2012_group.reset_index(inplace=True)

# Dati 2018
forest_surface_2018 = focus_2018_group[focus_2018_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_2018_group = pd.DataFrame(forest_surface_2018.groupby(['pro_com', 'comune', 'sup_com_ha'])['surface_ha_2018'].sum())
forest_surface_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_forest = [forest_surface_2018_group, forest_surface_2012_group, forest_surface_2006_group, forest_surface_2000_group, forest_surface_1990_group]
focus_forest_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_forest)
focus_forest_df = focus_forest_df[focus_forest_df['sup_com_ha'] > 0]
focus_forest_df.fillna(0, inplace=True)
focus_forest_df.insert(loc=3, column='perc_1990', value=round(focus_forest_df.surface_ha_1990 / focus_forest_df.sup_com_ha, 4) * 100)
focus_forest_df.insert(loc=4, column='perc_2000', value=round(focus_forest_df.surface_ha_2000 / focus_forest_df.sup_com_ha, 4) * 100)
focus_forest_df.insert(loc=5, column='perc_2006', value=round(focus_forest_df.surface_ha_2006 / focus_forest_df.sup_com_ha, 4) * 100)
focus_forest_df.insert(loc=6, column='perc_2012', value=round(focus_forest_df.surface_ha_2012 / focus_forest_df.sup_com_ha, 4) * 100)
focus_forest_df.insert(loc=7, column='perc_2018', value=round(focus_forest_df.surface_ha_2018 / focus_forest_df.sup_com_ha, 4) * 100)
focus_forest_df.insert(loc=8, column='Perc.Diff. | 2018-2000', value=(focus_forest_df.perc_2018 - focus_forest_df.perc_2000))
focus_forest_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
focus_forest_df

In [ ]:
# Superfici in arretramento
reduction_forest_area = focus_forest_df.iloc[0:10]
reduction_forest_area

In [ ]:
# Superfici in avanzamento
increase_forest_area = focus_forest_df.iloc[-10:]
increase_forest_area.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
increase_forest_area

In [ ]:
forest_df = pd.concat([reduction_forest_area, increase_forest_area])
forest_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
forest_df = forest_df[['comune', 'Perc.Diff. | 2018-2000']]
forest_df['Perc.Diff. | 2018-2000'] = forest_df['Perc.Diff. | 2018-2000']
forest_df.set_index('comune', inplace=True)

fig_forest = plt.figure(figsize=(20, 10), dpi=100)
sub_fig_forest = GridSpec(nrows=1, ncols=1, figure=fig_forest, hspace=0.35)
ax_forest = fig_forest.add_subplot(sub_fig_forest[0, 0])
forest_df.plot.bar(ax=ax_forest, rot=0)

ax_forest.yaxis.set_label_text('%')
ax_forest.set_xlabel('')
ax_forest.grid(color='lightgrey', linestyle='dashdot')

plt.xticks(rotation = 90)
plt.legend(loc='best')

E' significativo per me che nella top 10 del decremento di aree forestate ci siano Terzigno, Ercolano e Trecase. Lo è perchè sono tra i Comuni ricadenti nell'area del Parco Nazionale del Vesuvio che hanno subito danni al patrimonio boschivo a causa dell'[incendio che ci fu nel 2017](https://massimilianomoraca.it/blog/gis/analisi-temporale-sullarea-degli-incendi-del-vesuvio/).

Le aree in cui i boschi avanzano sono perlopiu aree rurali, a vocazione agricola, che pian piano si stanno spopolando.

### 2.3.3 Aree agricole
Il gruppo 2.1 della CLC

In [ ]:

# Dati 1990
arable_lands_surface_1990 = focus_1990_group[focus_1990_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_1990_group = pd.DataFrame(arable_lands_surface_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
arable_lands_surface_1990_group.reset_index(inplace=True)

# Dati 2000
arable_lands_surface_2000 = focus_2000_group[focus_2000_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_2000_group = pd.DataFrame(arable_lands_surface_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
arable_lands_surface_2000_group.reset_index(inplace=True)

# Dati 2006
arable_lands_surface_2006 = focus_2006_group[focus_2006_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_2006_group = pd.DataFrame(arable_lands_surface_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
arable_lands_surface_2006_group.reset_index(inplace=True)

# Dati 2012
arable_lands_surface_2012 = focus_2012_group[focus_2012_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_2012_group = pd.DataFrame(arable_lands_surface_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
arable_lands_surface_2012_group.reset_index(inplace=True)

# Dati 2018
arable_lands_surface_2018 = focus_2018_group[focus_2018_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_2018_group = pd.DataFrame(arable_lands_surface_2018.groupby(['pro_com', 'comune', 'sup_com_ha'])['surface_ha_2018'].sum())
arable_lands_surface_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_arable_lands = [arable_lands_surface_2018_group, arable_lands_surface_2012_group, arable_lands_surface_2006_group, arable_lands_surface_2000_group, arable_lands_surface_1990_group]
focus_arable_lands_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_arable_lands)
focus_arable_lands_df = focus_arable_lands_df[focus_arable_lands_df['sup_com_ha'] > 0]
focus_arable_lands_df.fillna(0, inplace=True)
focus_arable_lands_df.insert(loc=3, column='perc_1990', value=round(focus_arable_lands_df.surface_ha_1990 / focus_arable_lands_df.sup_com_ha, 4) * 100)
focus_arable_lands_df.insert(loc=4, column='perc_2000', value=round(focus_arable_lands_df.surface_ha_2000 / focus_arable_lands_df.sup_com_ha, 4) * 100)
focus_arable_lands_df.insert(loc=5, column='perc_2006', value=round(focus_arable_lands_df.surface_ha_2006 / focus_arable_lands_df.sup_com_ha, 4) * 100)
focus_arable_lands_df.insert(loc=6, column='perc_2012', value=round(focus_arable_lands_df.surface_ha_2012 / focus_arable_lands_df.sup_com_ha, 4) * 100)
focus_arable_lands_df.insert(loc=7, column='perc_2018', value=round(focus_arable_lands_df.surface_ha_2018 / focus_arable_lands_df.sup_com_ha, 4) * 100)
focus_arable_lands_df.insert(loc=8, column='Perc.Diff. | 2018-2000', value=(focus_arable_lands_df.perc_2018 - focus_arable_lands_df.perc_2000))
focus_arable_lands_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
focus_arable_lands_df

In [ ]:

# Superfici in arretramento
reduction_arable_lands_area = focus_arable_lands_df.iloc[0:10]
reduction_arable_lands_area

In [ ]:

# Superfici in avanzamento
increase_arable_lands_area = focus_arable_lands_df.iloc[-10:]
increase_arable_lands_area.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
increase_arable_lands_area

In [ ]:
arable_lands_df = pd.concat([reduction_arable_lands_area, increase_arable_lands_area])
arable_lands_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
arable_lands_df = arable_lands_df[['comune', 'Perc.Diff. | 2018-2000']]
arable_lands_df['Perc.Diff. | 2018-2000'] = arable_lands_df['Perc.Diff. | 2018-2000']
arable_lands_df.set_index('comune', inplace=True)

fig_arable_lands = plt.figure(figsize=(20, 10), dpi=100)
sub_fig_arable_lands = GridSpec(nrows=1, ncols=1, figure=fig_arable_lands, hspace=0.35)
ax_arable_lands = fig_arable_lands.add_subplot(sub_fig_arable_lands[0, 0])
arable_lands_df.plot.bar(ax=ax_arable_lands, rot=0)

ax_arable_lands.yaxis.set_label_text('%')
ax_arable_lands.set_xlabel('')
ax_arable_lands.grid(color='lightgrey', linestyle='dashdot')

plt.xticks(rotation = 90)
plt.legend(loc='best')

Tra i Comuni con maggior aumento della aree articifiali abbiamo visto che c'è Volla che è tra i Comuni con maggior decremento delle aree agricole. Poco sopra ho fatto una riflessione sul perchè della redistribuzione del livello 1 della CLC. **Due indizi non fanno una prova, ma tre?**

### 2.3.4 Aree bruciate


In [ ]:

# Dati 1990
burnt_surface_1990 = focus_1990_group[focus_1990_group['label3'].isin(['Burnt areas'])]
burnt_surface_1990_group = pd.DataFrame(burnt_surface_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
burnt_surface_1990_group.reset_index(inplace=True)

# Dati 2000
burnt_surface_2000 = focus_2000_group[focus_2000_group['label3'].isin(['Burnt areas'])]
burnt_surface_2000_group = pd.DataFrame(burnt_surface_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
burnt_surface_2000_group.reset_index(inplace=True)

# Dati 2006
burnt_surface_2006 = focus_2006_group[focus_2006_group['label3'].isin(['Burnt areas'])]
burnt_surface_2006_group = pd.DataFrame(burnt_surface_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
burnt_surface_2006_group.reset_index(inplace=True)

# Dati 2012
burnt_surface_2012 = focus_2012_group[focus_2012_group['label3'].isin(['Burnt areas'])]
burnt_surface_2012_group = pd.DataFrame(burnt_surface_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
burnt_surface_2012_group.reset_index(inplace=True)

# Dati 2018
burnt_surface_2018 = focus_2018_group[focus_2018_group['label3'].isin(['Burnt areas'])]
burnt_surface_2018_group = pd.DataFrame(burnt_surface_2018.groupby(['pro_com', 'comune', 'sup_com_ha'])['surface_ha_2018'].sum())
burnt_surface_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_arable_lands = [burnt_surface_2018_group, burnt_surface_2012_group, burnt_surface_2006_group, burnt_surface_2000_group, burnt_surface_1990_group]
focus_burnt_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_arable_lands)
focus_burnt_df = focus_burnt_df[focus_burnt_df['sup_com_ha'] > 0]
focus_burnt_df.fillna(0, inplace=True)
focus_burnt_df.insert(loc=3, column='perc_1990', value=round(focus_burnt_df.surface_ha_1990 / focus_burnt_df.sup_com_ha, 4) * 100)
focus_burnt_df.insert(loc=4, column='perc_2000', value=round(focus_burnt_df.surface_ha_2000 / focus_burnt_df.sup_com_ha, 4) * 100)
focus_burnt_df.insert(loc=5, column='perc_2006', value=round(focus_burnt_df.surface_ha_2006 / focus_burnt_df.sup_com_ha, 4) * 100)
focus_burnt_df.insert(loc=6, column='perc_2012', value=round(focus_burnt_df.surface_ha_2012 / focus_burnt_df.sup_com_ha, 4) * 100)
focus_burnt_df.insert(loc=7, column='perc_2018', value=round(focus_burnt_df.surface_ha_2018 / focus_burnt_df.sup_com_ha, 4) * 100)
focus_burnt_df.insert(loc=8, column='Perc.Diff. | 2018-2000', value=(focus_burnt_df.perc_2018 - focus_burnt_df.perc_2000))
focus_burnt_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
focus_burnt_df

In questa analisi ci sono in 6 delle prime 10 posizioni alcuni dei Comuni interessati dall'incendio del 2017.

### 2.3.5 Aree adibite a discarica

In [ ]:

# Dati 1990
dump_surface_1990 = focus_1990_group[focus_1990_group['label3'].isin(['Dump sites'])]
dump_surface_1990_group = pd.DataFrame(dump_surface_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
dump_surface_1990_group.reset_index(inplace=True)

# Dati 2000
dump_surface_2000 = focus_2000_group[focus_2000_group['label3'].isin(['Dump sites'])]
dump_surface_2000_group = pd.DataFrame(dump_surface_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
dump_surface_2000_group.reset_index(inplace=True)

# Dati 2006
dump_surface_2006 = focus_2006_group[focus_2006_group['label3'].isin(['Dump sites'])]
dump_surface_2006_group = pd.DataFrame(dump_surface_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
dump_surface_2006_group.reset_index(inplace=True)

# Dati 2012
dump_surface_2012 = focus_2012_group[focus_2012_group['label3'].isin(['Dump sites'])]
dump_surface_2012_group = pd.DataFrame(dump_surface_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
dump_surface_2012_group.reset_index(inplace=True)

# Dati 2018
dump_surface_2018 = focus_2018_group[focus_2018_group['label3'].isin(['Dump sites'])]
dump_surface_2018_group = pd.DataFrame(dump_surface_2018.groupby(['pro_com', 'comune', 'sup_com_ha'])['surface_ha_2018'].sum())
dump_surface_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_arable_lands = [dump_surface_2018_group, dump_surface_2012_group, dump_surface_2006_group, dump_surface_2000_group, dump_surface_1990_group]
focus_dump_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_arable_lands)
focus_dump_df = focus_dump_df[focus_dump_df['sup_com_ha'] > 0]
focus_dump_df.fillna(0, inplace=True)
focus_dump_df.insert(loc=3, column='perc_1990', value=round(focus_dump_df.surface_ha_1990 / focus_dump_df.sup_com_ha, 4) * 100)
focus_dump_df.insert(loc=4, column='perc_2000', value=round(focus_dump_df.surface_ha_2000 / focus_dump_df.sup_com_ha, 4) * 100)
focus_dump_df.insert(loc=5, column='perc_2006', value=round(focus_dump_df.surface_ha_2006 / focus_dump_df.sup_com_ha, 4) * 100)
focus_dump_df.insert(loc=6, column='perc_2012', value=round(focus_dump_df.surface_ha_2012 / focus_dump_df.sup_com_ha, 4) * 100)
focus_dump_df.insert(loc=7, column='perc_2018', value=round(focus_dump_df.surface_ha_2018 / focus_dump_df.sup_com_ha, 4) * 100)
focus_dump_df.insert(loc=8, column='Perc.Diff. | 2018-2000', value=(focus_dump_df.perc_2018 - focus_dump_df.perc_2000))
focus_dump_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
focus_dump_df

Qui abbiamo le più grandi discariche campane, cresciute con la crisi dei rifiuti di inizio anni 2000.

### 2.3.6 Aree verdi urbane

In [ ]:
# Dati 1990
green_urban_surface_1990 = focus_1990_group[focus_1990_group['label3'].isin(['Green urban areas'])]
green_urban_surface_1990_group = pd.DataFrame(green_urban_surface_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
green_urban_surface_1990_group.reset_index(inplace=True)

# Dati 2000
green_urban_surface_2000 = focus_2000_group[focus_2000_group['label3'].isin(['Green urban areas'])]
green_urban_surface_2000_group = pd.DataFrame(green_urban_surface_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
green_urban_surface_2000_group.reset_index(inplace=True)

# Dati 2006
green_urban_surface_2006 = focus_2006_group[focus_2006_group['label3'].isin(['Green urban areas'])]
green_urban_surface_2006_group = pd.DataFrame(green_urban_surface_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
green_urban_surface_2006_group.reset_index(inplace=True)

# Dati 2012
green_urban_surface_2012 = focus_2012_group[focus_2012_group['label3'].isin(['Green urban areas'])]
green_urban_surface_2012_group = pd.DataFrame(green_urban_surface_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
green_urban_surface_2012_group.reset_index(inplace=True)

# Dati 2018
green_urban_surface_2018 = focus_2018_group[focus_2018_group['label3'].isin(['Green urban areas'])]
green_urban_surface_2018_group = pd.DataFrame(green_urban_surface_2018.groupby(['pro_com', 'comune', 'sup_com_ha'])['surface_ha_2018'].sum())
green_urban_surface_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_green_urban = [green_urban_surface_2018_group, green_urban_surface_2012_group, green_urban_surface_2006_group, green_urban_surface_2000_group, green_urban_surface_1990_group]
focus_green_urban_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_green_urban)
focus_green_urban_df = focus_green_urban_df[focus_green_urban_df['sup_com_ha'] > 0]
focus_green_urban_df.fillna(0, inplace=True)
focus_green_urban_df.insert(loc=3, column='perc_1990', value=round(focus_green_urban_df.surface_ha_1990 / focus_green_urban_df.sup_com_ha, 4) * 100)
focus_green_urban_df.insert(loc=4, column='perc_2000', value=round(focus_green_urban_df.surface_ha_2000 / focus_green_urban_df.sup_com_ha, 4) * 100)
focus_green_urban_df.insert(loc=5, column='perc_2006', value=round(focus_green_urban_df.surface_ha_2006 / focus_green_urban_df.sup_com_ha, 4) * 100)
focus_green_urban_df.insert(loc=6, column='perc_2012', value=round(focus_green_urban_df.surface_ha_2012 / focus_green_urban_df.sup_com_ha, 4) * 100)
focus_green_urban_df.insert(loc=7, column='perc_2018', value=round(focus_green_urban_df.surface_ha_2018 / focus_green_urban_df.sup_com_ha, 4) * 100)
focus_green_urban_df.insert(loc=8, column='Perc.Diff. | 2018-2000', value=(focus_green_urban_df.perc_2018 - focus_green_urban_df.perc_2000))
focus_green_urban_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
focus_green_urban_df

# 3. Analisi a scala nazionale
Fino a qui abbiamo visto una analisi focalizzata sulla Campania. Di seguito ripeterò tutte le analisi ma a scala nazionale.


### 3.1.1 Stato al 1990

In [ ]:
clc1990_label1 = pd.DataFrame(clc1990.groupby('label1')['surface_ha_1990'].sum())
clc1990_label1.reset_index(inplace=True)
clc1990_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)
clc1990_label1

In [ ]:
clc1990_label2 = pd.DataFrame(clc1990.groupby('label2')['surface_ha_1990'].sum())
clc1990_label2.reset_index(inplace=True)
clc1990_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)
clc1990_label2

In [ ]:
clc1990_label3 = pd.DataFrame(clc1990.groupby('label3')['surface_ha_1990'].sum())
clc1990_label3.reset_index(inplace=True)
clc1990_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)
clc1990_label3

### 3.1.2 Stato al 2000

In [ ]:
clc2000_label1 = pd.DataFrame(clc2000.groupby('label1')['surface_ha_2000'].sum())
clc2000_label1.reset_index(inplace=True)
clc2000_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)

clc2000_label2 = pd.DataFrame(clc2000.groupby('label2')['surface_ha_2000'].sum())
clc2000_label2.reset_index(inplace=True)
clc2000_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)

clc2000_label3 = pd.DataFrame(clc2000.groupby('label3')['surface_ha_2000'].sum())
clc2000_label3.reset_index(inplace=True)
clc2000_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)

### 3.1.3 Stato al 2006

In [ ]:
clc2006_label1 = pd.DataFrame(clc2006.groupby('label1')['surface_ha_2006'].sum())
clc2006_label1.reset_index(inplace=True)
clc2006_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)

clc2006_label2 = pd.DataFrame(clc2006.groupby('label2')['surface_ha_2006'].sum())
clc2006_label2.reset_index(inplace=True)
clc2006_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)

clc2006_label3 = pd.DataFrame(clc2006.groupby('label3')['surface_ha_2006'].sum())
clc2006_label3.reset_index(inplace=True)
clc2006_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)

### 3.1.4 Stato al 2012

In [ ]:
clc2012_label1 = pd.DataFrame(clc2012.groupby('label1')['surface_ha_2012'].sum())
clc2012_label1.reset_index(inplace=True)
clc2012_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)

clc2012_label2 = pd.DataFrame(clc2012.groupby('label2')['surface_ha_2012'].sum())
clc2012_label2.reset_index(inplace=True)
clc2012_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)

clc2012_label3 = pd.DataFrame(clc2012.groupby('label3')['surface_ha_2012'].sum())
clc2012_label3.reset_index(inplace=True)
clc2012_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)

### 3.1.5 Stato al 2018

In [ ]:
clc2018_label1 = pd.DataFrame(clc2018.groupby('label1')['surface_ha_2018'].sum())
clc2018_label1.reset_index(inplace=True)
clc2018_label1.rename(columns={'label1': 'CORINE Land Cover - level 1'}, inplace=True)

clc2018_label2 = pd.DataFrame(clc2018.groupby('label2')['surface_ha_2018'].sum())
clc2018_label2.reset_index(inplace=True)
clc2018_label2.rename(columns={'label2': 'CORINE Land Cover - level 2'}, inplace=True)

clc2018_label3 = pd.DataFrame(clc2018.groupby('label3')['surface_ha_2018'].sum())
clc2018_label3.reset_index(inplace=True)
clc2018_label3.rename(columns={'label3': 'CORINE Land Cover - level 3'}, inplace=True)

## 3.2 Grafici
Mi limiterò a commentare solo la analisi a scala nazionale perchè quelle a scala comunale hanno risultati che, sebbene intuibili dal punto di vista delle variazioni, hanno una valenza territoriale molto forte(come lo è stato per la Campania) e non conoscendo tutti quei territori preferisco lasciare a chi è del posto le considerazioni del caso.


### 3.2.1 Grafici per il livello 1

In [ ]:
lvl1_naz = [clc1990_label1, clc2000_label1, clc2006_label1, clc2012_label1, clc2018_label1]
lvl1_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['CORINE Land Cover - level 1'], how='outer'), lvl1_naz)
lvl1_naz_df.insert(loc=6, column='Surf.Diff. | 2018-1990', value=(lvl1_naz_df.surface_ha_2018 - lvl1_naz_df.surface_ha_1990))
lvl1_naz_df.fillna(0, inplace=True)
lvl1_naz_df.set_index('CORINE Land Cover - level 1', inplace=True)
lvl1_naz_df.rename(columns={'surface_ha_1990': '1990', 'surface_ha_2000': '2000', 'surface_ha_2006': '2006', 'surface_ha_2012': '2012', 'surface_ha_2018': '2018'}, inplace=True)
lvl1_naz_df

In [ ]:
fig1_naz = plt.figure(figsize=(20, 10), dpi=100)
sub_fig_naz = GridSpec(nrows=1, ncols=1, figure=fig1_naz, hspace=0.35)
ax_naz = fig1_naz.add_subplot(sub_fig_naz[0, 0])
lvl1_naz_df.plot.bar(ax=ax_naz, rot=0)

ax_naz.yaxis.set_label_text('ha')
ax_naz.set_xlabel('')

plt.legend(loc='best')


In questo livello di CLC si nota una certa somiglianza con ciò che abbiamo visto per la Campania: aree artificiali in aumento, aree forestate ed agricole in regressione.

### 3.2.2 Grafici per il livello 2

In [ ]:
lvl2_naz = [clc1990_label2, clc2000_label2, clc2006_label2, clc2012_label2, clc2018_label2]
lvl2_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['CORINE Land Cover - level 2'], how='outer'), lvl2_naz)
lvl2_naz_df.insert(loc=6, column='Surf.Diff. | 2000-1990', value=(lvl2_naz_df.surface_ha_2000 - lvl2_naz_df.surface_ha_1990))
lvl2_naz_df.insert(loc=7, column='Surf.Diff. | 2006-2000', value=(lvl2_naz_df.surface_ha_2006 - lvl2_naz_df.surface_ha_2000))
lvl2_naz_df.insert(loc=8, column='Surf.Diff. | 2012-2006', value=(lvl2_naz_df.surface_ha_2012 - lvl2_naz_df.surface_ha_2006))
lvl2_naz_df.insert(loc=9, column='Surf.Diff. | 2018-2012', value=(lvl2_naz_df.surface_ha_2018 - lvl2_naz_df.surface_ha_2012))
lvl2_naz_df.insert(loc=9, column='Surf.Diff. | 2018-1990', value=(lvl2_naz_df.surface_ha_2018 - lvl2_naz_df.surface_ha_1990))
lvl2_naz_df.fillna(0, inplace=True)
lvl2_naz_df.set_index('CORINE Land Cover - level 2', inplace=True)
lvl2_naz_df = lvl2_naz_df[['Surf.Diff. | 2000-1990', 'Surf.Diff. | 2006-2000', 'Surf.Diff. | 2012-2006', 'Surf.Diff. | 2018-2012', 'Surf.Diff. | 2018-1990']]
lvl2_naz_df

In [ ]:
fig2_naz = plt.figure(figsize=(20, 10), dpi=100)
sub_fig_naz = GridSpec(nrows=1, ncols=1, figure=fig2_naz, hspace=0.35)
ax_naz = fig2_naz.add_subplot(sub_fig_naz[0, 0])
lvl2_naz_df.plot.bar(ax=ax_naz, rot=0)

ax_naz.yaxis.set_label_text('ha')
ax_naz.set_xlabel('')

plt.xticks(rotation = 90)
plt.legend(loc='best')

Anche qui c'è una certa somiglianza con ciò che accade in Campania.

### 3.2.3 Grafici per il livello 3

In [ ]:
lvl3_naz = [clc1990_label3, clc2000_label3, clc2006_label3, clc2012_label3, clc2018_label3]
lvl3_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['CORINE Land Cover - level 3'], how='outer'), lvl3_naz)
lvl3_naz_df.insert(loc=6, column='Surf.Diff. | 2000-1990', value=(lvl3_naz_df.surface_ha_2000 - lvl3_naz_df.surface_ha_1990))
lvl3_naz_df.insert(loc=7, column='Surf.Diff. | 2006-2000', value=(lvl3_naz_df.surface_ha_2006 - lvl3_naz_df.surface_ha_2000))
lvl3_naz_df.insert(loc=8, column='Surf.Diff. | 2012-2006', value=(lvl3_naz_df.surface_ha_2012 - lvl3_naz_df.surface_ha_2006))
lvl3_naz_df.insert(loc=9, column='Surf.Diff. | 2018-2012', value=(lvl3_naz_df.surface_ha_2018 - lvl3_naz_df.surface_ha_2012))
lvl3_naz_df.insert(loc=9, column='Surf.Diff. | 2018-1990', value=(lvl3_naz_df.surface_ha_2018 - lvl3_naz_df.surface_ha_1990))
lvl3_naz_df.set_index('CORINE Land Cover - level 3', inplace=True)
lvl3_naz_df.fillna(0, inplace=True)
lvl3_naz_df

In [ ]:
fig3_naz = plt.figure(figsize=(20, 10), dpi=100)
sub_fig_naz = GridSpec(nrows=1, ncols=1, figure=fig3_naz, hspace=0.35)
ax_naz = fig3_naz.add_subplot(sub_fig_naz[0, 0])
lvl3_df['Surf.Diff. | 2018-1990'].plot.bar(ax=ax_naz, rot=0)

ax_naz.yaxis.set_label_text('ha')
ax_naz.set_xlabel('')

plt.xticks(rotation = 90)
plt.legend(loc='best')

Purtroppo il *"trend campano"* è visibile anche qui. Il fatto è che ciò che accade a scala nazionale si ripete a scala regionale e scommetto anche provinciale in un po' tutto il territorio italiano.

## 3.3 Focus sui Comuni

In [ ]:
# Dati 1990
focus_1990_naz = clc1990[['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3', 'surface_ha_1990']]
focus_1990_naz.set_index('label3', inplace=True)
focus_naz_1990_group = pd.DataFrame(focus_1990_naz.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3'])['surface_ha_1990'].sum())
focus_naz_1990_group.reset_index(inplace=True)

# Dati 2000
focus_2000_naz = clc2000[['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3', 'surface_ha_2000']]
focus_2000_naz.set_index('label3', inplace=True)
focus_naz_2000_group = pd.DataFrame(focus_2000_naz.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3'])['surface_ha_2000'].sum())
focus_naz_2000_group.reset_index(inplace=True)

# Dati 2006
focus_2006_naz = clc2006[['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3', 'surface_ha_2006']]
focus_2006_naz.set_index('label3', inplace=True)
focus_naz_2006_group = pd.DataFrame(focus_2006_naz.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3'])['surface_ha_2006'].sum())
focus_naz_2006_group.reset_index(inplace=True)

# Dati 2012
focus_2012_naz = clc2012[['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3', 'surface_ha_2012']]
focus_2012_naz.set_index('label3', inplace=True)
focus_naz_2012_group = pd.DataFrame(focus_2012_naz.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3'])['surface_ha_2012'].sum())
focus_naz_2012_group.reset_index(inplace=True)

# Dati 2018
focus_2018_naz = clc2018[['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3', 'surface_ha_2018']]
focus_2018_naz.set_index('label3', inplace=True)
focus_naz_2018_group = pd.DataFrame(focus_2018_naz.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha', 'label3'])['surface_ha_2018'].sum())
focus_naz_2018_group.reset_index(inplace=True)

In [ ]:
focus_naz_2018_group

### 3.3.1 Aree artificiali
I gruppi 1.1 ed 1.2 della CLC

In [ ]:
# Dati 1990
artificial_surface_naz_1990 = focus_naz_1990_group[focus_naz_1990_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_naz_1990_group = pd.DataFrame(artificial_surface_naz_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
artificial_surface_naz_1990_group.reset_index(inplace=True)

# Dati 2000
artificial_surface_naz_2000 = focus_naz_2000_group[focus_naz_2000_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_naz_2000_group = pd.DataFrame(artificial_surface_naz_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
artificial_surface_naz_2000_group.reset_index(inplace=True)

# Dati 2006
artificial_surface_naz_2006 = focus_naz_2006_group[focus_naz_2006_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_naz_2006_group = pd.DataFrame(artificial_surface_naz_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
artificial_surface_naz_2006_group.reset_index(inplace=True)

# Dati 2012
artificial_surface_naz_2012 = focus_naz_2012_group[focus_naz_2012_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_naz_2012_group = pd.DataFrame(artificial_surface_naz_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
artificial_surface_naz_2012_group.reset_index(inplace=True)

# Dati 2018
artificial_surface_naz_2018 = focus_naz_2018_group[focus_naz_2018_group['label3'].isin(['Continuous urban fabric', 'Discontinuous urban fabric', 'Industrial or commercial units', 'Road and rail networks and associated land', 'Port areas', 'Airports'])]
artificial_surface_naz_2018_group = pd.DataFrame(artificial_surface_naz_2018.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha'])['surface_ha_2018'].sum())
artificial_surface_naz_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_artificial_naz = [artificial_surface_naz_2018_group, artificial_surface_naz_2012_group, artificial_surface_naz_2006_group, artificial_surface_naz_2000_group, artificial_surface_naz_1990_group]
focus_artificial_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_artificial_naz)
focus_artificial_naz_df = focus_artificial_naz_df[focus_artificial_naz_df['sup_com_ha'] > 0]
focus_artificial_naz_df.fillna(0, inplace=True)
focus_artificial_naz_df.insert(loc=5, column='perc_1990', value=round(focus_artificial_naz_df.surface_ha_1990 / focus_artificial_naz_df.sup_com_ha, 4) * 100)
focus_artificial_naz_df.insert(loc=6, column='perc_2000', value=round(focus_artificial_naz_df.surface_ha_2000 / focus_artificial_naz_df.sup_com_ha, 4) * 100)
focus_artificial_naz_df.insert(loc=7, column='perc_2006', value=round(focus_artificial_naz_df.surface_ha_2006 / focus_artificial_naz_df.sup_com_ha, 4) * 100)
focus_artificial_naz_df.insert(loc=8, column='perc_2012', value=round(focus_artificial_naz_df.surface_ha_2012 / focus_artificial_naz_df.sup_com_ha, 4) * 100)
focus_artificial_naz_df.insert(loc=9, column='perc_2018', value=round(focus_artificial_naz_df.surface_ha_2018 / focus_artificial_naz_df.sup_com_ha, 4) * 100)
focus_artificial_naz_df.insert(loc=10, column='Perc.Diff. | 2018-2000', value=(focus_artificial_naz_df.perc_2018 - focus_artificial_naz_df.perc_2000))
focus_artificial_naz_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
focus_artificial_naz_df

In [ ]:
# Superfici in arretramento
naz_reduction_artificial_area = focus_artificial_naz_df.iloc[0:10]
naz_reduction_artificial_area

In [ ]:
# Superfici in avanzamento
naz_increase_artificial_area = focus_artificial_naz_df.iloc[-10:]
naz_increase_artificial_area.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
naz_increase_artificial_area

### 3.3.2 Aree forestate
Il gruppo 3.1 della CLC

In [ ]:
# Dati 1990
forest_surface_naz_1990 = focus_naz_1990_group[focus_naz_1990_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_naz_1990_group = pd.DataFrame(forest_surface_naz_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
forest_surface_naz_1990_group.reset_index(inplace=True)

# Dati 2000
forest_surface_naz_2000 = focus_naz_2000_group[focus_naz_2000_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_naz_2000_group = pd.DataFrame(forest_surface_naz_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
forest_surface_naz_2000_group.reset_index(inplace=True)

# Dati 2006
forest_surface_naz_2006 = focus_naz_2006_group[focus_naz_2006_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_naz_2006_group = pd.DataFrame(forest_surface_naz_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
forest_surface_naz_2006_group.reset_index(inplace=True)

# Dati 2012
forest_surface_naz_2012 = focus_naz_2012_group[focus_naz_2012_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_naz_2012_group = pd.DataFrame(forest_surface_naz_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
forest_surface_naz_2012_group.reset_index(inplace=True)

# Dati 2018
forest_surface_naz_2018 = focus_naz_2018_group[focus_naz_2018_group['label3'].isin(['Broad-leaved forest', 'Coniferous forest', 'Mixed forest'])]
forest_surface_naz_2018_group = pd.DataFrame(forest_surface_naz_2018.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha'])['surface_ha_2018'].sum())
forest_surface_naz_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_forest_naz = [forest_surface_naz_2018_group, forest_surface_naz_2012_group, forest_surface_naz_2006_group, forest_surface_naz_2000_group, forest_surface_naz_1990_group]
focus_forest_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_forest_naz)
focus_forest_naz_df = focus_forest_naz_df[focus_forest_naz_df['sup_com_ha'] > 0]
focus_forest_naz_df.fillna(0, inplace=True)
focus_forest_naz_df.insert(loc=5, column='perc_1990', value=round(focus_forest_naz_df.surface_ha_1990 / focus_forest_naz_df.sup_com_ha, 4) * 100)
focus_forest_naz_df.insert(loc=6, column='perc_2000', value=round(focus_forest_naz_df.surface_ha_2000 / focus_forest_naz_df.sup_com_ha, 4) * 100)
focus_forest_naz_df.insert(loc=7, column='perc_2006', value=round(focus_forest_naz_df.surface_ha_2006 / focus_forest_naz_df.sup_com_ha, 4) * 100)
focus_forest_naz_df.insert(loc=8, column='perc_2012', value=round(focus_forest_naz_df.surface_ha_2012 / focus_forest_naz_df.sup_com_ha, 4) * 100)
focus_forest_naz_df.insert(loc=9, column='perc_2018', value=round(focus_forest_naz_df.surface_ha_2018 / focus_forest_naz_df.sup_com_ha, 4) * 100)
focus_forest_naz_df.insert(loc=10, column='Perc.Diff. | 2018-2000', value=(focus_forest_naz_df.perc_2018 - focus_forest_naz_df.perc_2000))
focus_forest_naz_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
focus_forest_naz_df

In [ ]:
# Superfici in arretramento
naz_reduction_forest_area = focus_forest_naz_df.iloc[0:10]
naz_reduction_forest_area

In [ ]:
# Superfici in avanzamento
naz_increase_forest_area = focus_forest_naz_df.iloc[-10:]
naz_increase_forest_area.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
naz_increase_forest_area

### 3.3.3 Aree agricole
Il gruppo 2.1 della CLC

In [ ]:
# Dati 1990
arable_lands_surface_naz_1990 = focus_naz_1990_group[focus_naz_1990_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_naz_1990_group = pd.DataFrame(arable_lands_surface_naz_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
arable_lands_surface_naz_1990_group.reset_index(inplace=True)

# Dati 2000
arable_lands_surface_naz_2000 = focus_naz_2000_group[focus_naz_2000_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_naz_2000_group = pd.DataFrame(arable_lands_surface_naz_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
arable_lands_surface_naz_2000_group.reset_index(inplace=True)

# Dati 2006
arable_lands_surface_naz_2006 = focus_naz_2006_group[focus_naz_2006_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_naz_2006_group = pd.DataFrame(arable_lands_surface_naz_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
arable_lands_surface_naz_2006_group.reset_index(inplace=True)

# Dati 2012
arable_lands_surface_naz_2012 = focus_naz_2012_group[focus_naz_2012_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_naz_2012_group = pd.DataFrame(arable_lands_surface_naz_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
arable_lands_surface_naz_2012_group.reset_index(inplace=True)

# Dati 2018
arable_lands_surface_naz_2018 = focus_naz_2018_group[focus_naz_2018_group['label3'].isin(['Non-irrigated arable land', 'Permanently irrigated land'])]
arable_lands_surface_naz_2018_group = pd.DataFrame(arable_lands_surface_naz_2018.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha'])['surface_ha_2018'].sum())
arable_lands_surface_naz_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_arable_lands_naz = [arable_lands_surface_naz_2018_group, arable_lands_surface_naz_2012_group, arable_lands_surface_naz_2006_group, arable_lands_surface_naz_2000_group, arable_lands_surface_naz_1990_group]
focus_arable_lands_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_arable_lands_naz)
focus_arable_lands_naz_df = focus_arable_lands_naz_df[focus_arable_lands_naz_df['sup_com_ha'] > 0]
focus_arable_lands_naz_df.fillna(0, inplace=True)
focus_arable_lands_naz_df.insert(loc=5, column='perc_1990', value=round(focus_arable_lands_naz_df.surface_ha_1990 / focus_arable_lands_naz_df.sup_com_ha, 4) * 100)
focus_arable_lands_naz_df.insert(loc=6, column='perc_2000', value=round(focus_arable_lands_naz_df.surface_ha_2000 / focus_arable_lands_naz_df.sup_com_ha, 4) * 100)
focus_arable_lands_naz_df.insert(loc=7, column='perc_2006', value=round(focus_arable_lands_naz_df.surface_ha_2006 / focus_arable_lands_naz_df.sup_com_ha, 4) * 100)
focus_arable_lands_naz_df.insert(loc=8, column='perc_2012', value=round(focus_arable_lands_naz_df.surface_ha_2012 / focus_arable_lands_naz_df.sup_com_ha, 4) * 100)
focus_arable_lands_naz_df.insert(loc=9, column='perc_2018', value=round(focus_arable_lands_naz_df.surface_ha_2018 / focus_arable_lands_naz_df.sup_com_ha, 4) * 100)
focus_arable_lands_naz_df.insert(loc=10, column='Perc.Diff. | 2018-2000', value=(focus_arable_lands_naz_df.perc_2018 - focus_arable_lands_naz_df.perc_2000))
focus_arable_lands_naz_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=True, inplace=True)
focus_arable_lands_naz_df

In [ ]:
# Superfici in arretramento
naz_reduction_arable_lands_area = focus_arable_lands_naz_df.iloc[0:10]
naz_reduction_arable_lands_area

In [ ]:
# Superfici in avanzamento
naz_increase_arable_lands_area = focus_arable_lands_naz_df.iloc[-10:]
naz_increase_arable_lands_area.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
naz_increase_arable_lands_area

### 3.3.4 Aree bruciate

In [ ]:
# Dati 1990
burnt_surface_naz_1990 = focus_naz_1990_group[focus_naz_1990_group['label3'].isin(['Burnt areas'])]
burnt_surface_naz_1990_group = pd.DataFrame(burnt_surface_naz_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
burnt_surface_naz_1990_group.reset_index(inplace=True)

# Dati 2000
burnt_surface_naz_2000 = focus_naz_2000_group[focus_naz_2000_group['label3'].isin(['Burnt areas'])]
burnt_surface_naz_2000_group = pd.DataFrame(burnt_surface_naz_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
burnt_surface_naz_2000_group.reset_index(inplace=True)

# Dati 2006
burnt_surface_naz_2006 = focus_naz_2006_group[focus_naz_2006_group['label3'].isin(['Burnt areas'])]
burnt_surface_naz_2006_group = pd.DataFrame(burnt_surface_naz_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
burnt_surface_naz_2006_group.reset_index(inplace=True)

# Dati 2012
burnt_surface_naz_2012 = focus_naz_2012_group[focus_naz_2012_group['label3'].isin(['Burnt areas'])]
burnt_surface_naz_2012_group = pd.DataFrame(burnt_surface_naz_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
burnt_surface_naz_2012_group.reset_index(inplace=True)

# Dati 2018
burnt_surface_naz_2018 = focus_naz_2018_group[focus_naz_2018_group['label3'].isin(['Burnt areas'])]
burnt_surface_naz_2018_group = pd.DataFrame(burnt_surface_naz_2018.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha'])['surface_ha_2018'].sum())
burnt_surface_naz_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_burnt_naz = [burnt_surface_naz_2018_group, burnt_surface_naz_2012_group, burnt_surface_naz_2006_group, burnt_surface_naz_2000_group, burnt_surface_naz_1990_group]
focus_burnt_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_burnt_naz)
focus_burnt_naz_df = focus_burnt_naz_df[focus_burnt_naz_df['sup_com_ha'] > 0]
focus_burnt_naz_df.fillna(0, inplace=True)
focus_burnt_naz_df.insert(loc=5, column='perc_1990', value=round(focus_burnt_naz_df.surface_ha_1990 / focus_burnt_naz_df.sup_com_ha, 4) * 100)
focus_burnt_naz_df.insert(loc=6, column='perc_2000', value=round(focus_burnt_naz_df.surface_ha_2000 / focus_burnt_naz_df.sup_com_ha, 4) * 100)
focus_burnt_naz_df.insert(loc=7, column='perc_2006', value=round(focus_burnt_naz_df.surface_ha_2006 / focus_burnt_naz_df.sup_com_ha, 4) * 100)
focus_burnt_naz_df.insert(loc=8, column='perc_2012', value=round(focus_burnt_naz_df.surface_ha_2012 / focus_burnt_naz_df.sup_com_ha, 4) * 100)
focus_burnt_naz_df.insert(loc=9, column='perc_2018', value=round(focus_burnt_naz_df.surface_ha_2018 / focus_burnt_naz_df.sup_com_ha, 4) * 100)
focus_burnt_naz_df.insert(loc=10, column='Perc.Diff. | 2018-2000', value=(focus_burnt_naz_df.perc_2018 - focus_burnt_naz_df.perc_2000))
focus_burnt_naz_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
focus_burnt_naz_df

### 3.3.5 Aree adibite a discarica

In [ ]:
# Dati 1990
dump_surface_naz_1990 = focus_naz_1990_group[focus_naz_1990_group['label3'].isin(['Dump sites'])]
dump_surface_naz_1990_group = pd.DataFrame(dump_surface_naz_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
dump_surface_naz_1990_group.reset_index(inplace=True)

# Dati 2000
dump_surface_naz_2000 = focus_naz_2000_group[focus_naz_2000_group['label3'].isin(['Dump sites'])]
dump_surface_naz_2000_group = pd.DataFrame(dump_surface_naz_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
dump_surface_naz_2000_group.reset_index(inplace=True)

# Dati 2006
dump_surface_naz_2006 = focus_naz_2006_group[focus_naz_2006_group['label3'].isin(['Dump sites'])]
dump_surface_naz_2006_group = pd.DataFrame(dump_surface_naz_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
dump_surface_naz_2006_group.reset_index(inplace=True)

# Dati 2012
dump_surface_naz_2012 = focus_naz_2012_group[focus_naz_2012_group['label3'].isin(['Dump sites'])]
dump_surface_naz_2012_group = pd.DataFrame(dump_surface_naz_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
dump_surface_naz_2012_group.reset_index(inplace=True)

# Dati 2018
dump_surface_naz_2018 = focus_naz_2018_group[focus_naz_2018_group['label3'].isin(['Dump sites'])]
dump_surface_naz_2018_group = pd.DataFrame(dump_surface_naz_2018.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha'])['surface_ha_2018'].sum())
dump_surface_naz_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_dump_naz = [dump_surface_naz_2018_group, dump_surface_naz_2012_group, dump_surface_naz_2006_group, dump_surface_naz_2000_group, dump_surface_naz_1990_group]
focus_dump_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_dump_naz)
focus_dump_naz_df = focus_dump_naz_df[focus_dump_naz_df['sup_com_ha'] > 0]
focus_dump_naz_df.fillna(0, inplace=True)
focus_dump_naz_df.insert(loc=5, column='perc_1990', value=round(focus_dump_naz_df.surface_ha_1990 / focus_dump_naz_df.sup_com_ha, 4) * 100)
focus_dump_naz_df.insert(loc=6, column='perc_2000', value=round(focus_dump_naz_df.surface_ha_2000 / focus_dump_naz_df.sup_com_ha, 4) * 100)
focus_dump_naz_df.insert(loc=7, column='perc_2006', value=round(focus_dump_naz_df.surface_ha_2006 / focus_dump_naz_df.sup_com_ha, 4) * 100)
focus_dump_naz_df.insert(loc=8, column='perc_2012', value=round(focus_dump_naz_df.surface_ha_2012 / focus_dump_naz_df.sup_com_ha, 4) * 100)
focus_dump_naz_df.insert(loc=9, column='perc_2018', value=round(focus_dump_naz_df.surface_ha_2018 / focus_dump_naz_df.sup_com_ha, 4) * 100)
focus_dump_naz_df.insert(loc=10, column='Perc.Diff. | 2018-2000', value=(focus_dump_naz_df.perc_2018 - focus_dump_naz_df.perc_2000))
focus_dump_naz_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
focus_dump_naz_df

In [ ]:
# Dati 1990
green_urban_surface_surface_naz_1990 = focus_naz_1990_group[focus_naz_1990_group['label3'].isin(['Green urban areas'])]
green_urban_surface_surface_naz_1990_group = pd.DataFrame(green_urban_surface_surface_naz_1990.groupby(['pro_com'])['surface_ha_1990'].sum())
green_urban_surface_surface_naz_1990_group.reset_index(inplace=True)

# Dati 2000
green_urban_surface_surface_naz_2000 = focus_naz_2000_group[focus_naz_2000_group['label3'].isin(['Green urban areas'])]
green_urban_surface_surface_naz_2000_group = pd.DataFrame(green_urban_surface_surface_naz_2000.groupby(['pro_com'])['surface_ha_2000'].sum())
green_urban_surface_surface_naz_2000_group.reset_index(inplace=True)

# Dati 2006
green_urban_surface_surface_naz_2006 = focus_naz_2006_group[focus_naz_2006_group['label3'].isin(['Green urban areas'])]
green_urban_surface_surface_naz_2006_group = pd.DataFrame(green_urban_surface_surface_naz_2006.groupby(['pro_com'])['surface_ha_2006'].sum())
green_urban_surface_surface_naz_2006_group.reset_index(inplace=True)

# Dati 2012
green_urban_surface_surface_naz_2012 = focus_naz_2012_group[focus_naz_2012_group['label3'].isin(['Green urban areas'])]
green_urban_surface_surface_naz_2012_group = pd.DataFrame(green_urban_surface_surface_naz_2012.groupby(['pro_com'])['surface_ha_2012'].sum())
green_urban_surface_surface_naz_2012_group.reset_index(inplace=True)

# Dati 2018
green_urban_surface_surface_naz_2018 = focus_naz_2018_group[focus_naz_2018_group['label3'].isin(['Green urban areas'])]
green_urban_surface_surface_naz_2018_group = pd.DataFrame(green_urban_surface_surface_naz_2018.groupby(['pro_com', 'comune', 'provincia', 'regione', 'sup_com_ha'])['surface_ha_2018'].sum())
green_urban_surface_surface_naz_2018_group.reset_index(inplace=True)

# Dati aggregati
focus_green_urban_surface_naz = [green_urban_surface_surface_naz_2018_group, green_urban_surface_surface_naz_2012_group, green_urban_surface_surface_naz_2006_group, green_urban_surface_surface_naz_2000_group, green_urban_surface_surface_naz_1990_group]
focus_green_urban_surface_naz_df = reduce(lambda  left,right: pd.merge(left,right,on=['pro_com'], how='outer'), focus_green_urban_surface_naz)
focus_green_urban_surface_naz_df = focus_green_urban_surface_naz_df[focus_green_urban_surface_naz_df['sup_com_ha'] > 0]
focus_green_urban_surface_naz_df.fillna(0, inplace=True)
focus_green_urban_surface_naz_df.insert(loc=5, column='perc_1990', value=round(focus_green_urban_surface_naz_df.surface_ha_1990 / focus_green_urban_surface_naz_df.sup_com_ha, 4) * 100)
focus_green_urban_surface_naz_df.insert(loc=6, column='perc_2000', value=round(focus_green_urban_surface_naz_df.surface_ha_2000 / focus_green_urban_surface_naz_df.sup_com_ha, 4) * 100)
focus_green_urban_surface_naz_df.insert(loc=7, column='perc_2006', value=round(focus_green_urban_surface_naz_df.surface_ha_2006 / focus_green_urban_surface_naz_df.sup_com_ha, 4) * 100)
focus_green_urban_surface_naz_df.insert(loc=8, column='perc_2012', value=round(focus_green_urban_surface_naz_df.surface_ha_2012 / focus_green_urban_surface_naz_df.sup_com_ha, 4) * 100)
focus_green_urban_surface_naz_df.insert(loc=9, column='perc_2018', value=round(focus_green_urban_surface_naz_df.surface_ha_2018 / focus_green_urban_surface_naz_df.sup_com_ha, 4) * 100)
focus_green_urban_surface_naz_df.insert(loc=10, column='Perc.Diff. | 2018-2000', value=(focus_green_urban_surface_naz_df.perc_2018 - focus_green_urban_surface_naz_df.perc_2000))
focus_green_urban_surface_naz_df.sort_values(by=['Perc.Diff. | 2018-2000'], ascending=False, inplace=True)
focus_green_urban_surface_naz_df

# Conclusione

Sembra abbastanza chiaro che in Italia negli ultimi trenta anni circa si sia preferito sacrificare il paesaggio agricolo e forestale in favore di quello urbano. Si parla molto di [Urban Heat Island](https://www.epa.gov/green-infrastructure/reduce-urban-heat-island-effect#:~:text=%22Urban%20heat%20islands%22%20occur%20when,heat%2Drelated%20illness%20and%20mortality.) e della [riduzione di questi fenomeni](https://www.sciencedirect.com/science/article/pii/S2210670721008301) tipici delle aree urbane. Magari, se si fosse riammodernato il costruito negli ultimi trenta anni al posto di edificare in nuovi luoghi, il fenomeno delle isole urbane di calore ci avrebbe colpito in maniera molto minore di quanto non ci colpisca adesso(e nei prossimi anni!). Avremmo così preservato il nostro patrimonio boschivo ed agricolo rendendo il *Bel Paese* ancora più bello.